# Regime-Aware Limit Order Book Analysis

## Overview

This notebook implements a regime-aware machine learning approach to predict short-horizon mid-price direction in cryptocurrency limit order books (LOB). The core hypothesis is that **market predictability is regime-dependent**: predictive signals are stronger during "toxic" market conditions (high order imbalance, aggressive cancellations) compared to "balanced" conditions.

## Pipeline

1. **Data Loading**: High-frequency BTC LOB data (1-second snapshots)
2. **Feature Engineering**: Order book depth, imbalances, market/cancel flows, realized volatility
3. **Label Creation**: Ternary classification (Up/Down/Flat) based on 30-second forward returns
4. **Regime Discovery**: Hidden Markov Model (HMM) to identify toxic vs. balanced market states
5. **Supervised Learning**: Logistic Regression and XGBoost with regime-aware features
6. **Walk-Forward Validation**: Time-series cross-validation with 9 folds
7. **Evaluation**: Regime-conditioned hit rates to validate hypothesis

## Key Hypothesis

**Toxic regimes** (characterized by large order imbalances and aggressive cancellation activity) should exhibit higher predictability than **balanced regimes** (symmetric order flow and stable depth). We measure this via "lift" = Hit Rate(toxic) - Hit Rate(balanced).

In [ ]:
import warnings, os, sys, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")

# Core ML libraries
from hmmlearn.hmm import GaussianHMM
from xgboost import XGBClassifier

# Try to import LightGBM (optional)
try:
    from lightgbm import LGBMClassifier
    LGBM_AVAILABLE = True
except ImportError:
    LGBM_AVAILABLE = False
    print("⚠ LightGBM not available. Install with: pip install lightgbm")

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import accuracy_score, f1_score

print("✓ All required libraries loaded successfully")
print(f"✓ LightGBM available: {LGBM_AVAILABLE}")

## 0. Configuration Parameters

This section defines all hyperparameters and settings for the pipeline. These values have been tuned for optimal performance on the BTC LOB dataset.

In [ ]:
# ===================================================================
# DATA & COLUMN CONFIGURATION
# ===================================================================
DATA_CSV = 'BTC_1sec.csv'  # High-frequency BTC LOB data from Kaggle

TIME_COL   = 'timestamp'   # Timestamp column name
MID_COL    = 'midpoint'    # Mid-price (average of best bid and ask)
SPREAD_COL = 'spread'      # Bid-ask spread

TOP_K = 15  # Number of order book levels to aggregate (depth)

# ===================================================================
# LABEL CREATION (BINARY CLASSIFICATION)
# ===================================================================
HORIZON_SEC = 30    # Prediction horizon in seconds (30s forward return)

# Threshold for up/down classification (basis points)
# BINARY classification: only predict UP vs DOWN (ignore small moves)
THRESH_BPS = 2.0    # 2 bp = 0.02% (moves smaller than this are filtered out)

# ===================================================================
# WALK-FORWARD VALIDATION
# ===================================================================
TRAIN_DAYS  = 3     # Days of historical data for training
TEST_HOURS  = 4     # Hours of data for testing per fold
EMBARGO_SEC = 60    # Embargo period between train and test (prevents leakage)
STRIDE_HRS  = 12    # Hours between consecutive folds (12hr = 9 folds total)

# ===================================================================
# PCA DIMENSIONALITY REDUCTION
# ===================================================================
USE_PCA = True       # Enable PCA for feature compression
PCA_VARIANCE = 0.95  # Retain 95% of variance (reduces 26 features to ~12-15 components)

# ===================================================================
# MODEL HYPERPARAMETERS
# ===================================================================
MAX_TRAIN_SAMPLES = 200000  # INCREASED from 120K for better model training

# HMM (Hidden Markov Model) for regime discovery (2 states: balanced vs toxic)
HMM_COVARIANCE = 'diag'     # Diagonal covariance (faster than 'full', works well in practice)
HMM_ITERATIONS = 150        # EM algorithm iterations (more = better convergence but slower)

# XGBoost (Gradient Boosting) for supervised prediction
XGB_N_ESTIMATORS = 300      # Number of boosting rounds
XGB_MAX_DEPTH = 6           # Maximum tree depth (controls model complexity)
XGB_LEARNING_RATE = 0.05    # Step size shrinkage (lower = more conservative, better generalization)
XGB_EARLY_STOPPING = 30     # Stop if validation score doesn't improve for 30 rounds

# ===================================================================
# REPRODUCIBILITY
# ===================================================================
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 1. Data Loading & Inspection

**Dataset**: High-frequency cryptocurrency LOB data from Kaggle ([link](https://www.kaggle.com/datasets/martinsn/high-frequency-crypto-limit-order-book-data))

The dataset contains 1-second snapshots of the Bitcoin limit order book with:
- **Timestamp**: System time for each snapshot
- **Midpoint**: (Best Bid + Best Ask) / 2
- **Spread**: Best Ask - Best Bid
- **Depth**: Notional values at each price level (bids/asks)
- **Flows**: Market orders, limit orders, and cancellations at each level

We load the data, standardize timestamps to UTC, and sort chronologically.

In [3]:

df = pd.read_csv(DATA_CSV)
# Basic normalization of time column
if TIME_COL not in df.columns:
    for c in df.columns:
        if 'time' in c.lower():
            TIME_COL = c
            break

df[TIME_COL] = pd.to_datetime(df[TIME_COL], utc=True, errors='coerce')
df = df.dropna(subset=[TIME_COL]).sort_values(TIME_COL).reset_index(drop=True)

print("Rows:", len(df))
print("Time range:", df[TIME_COL].min(), "→", df[TIME_COL].max())
print("Columns:", list(df.columns)[:12], "...")


Rows: 1030728
Time range: 2021-04-07 11:32:42.122161+00:00 → 2021-04-19 09:54:22.386544+00:00
Columns: ['Unnamed: 0', 'system_time', 'midpoint', 'spread', 'buys', 'sells', 'bids_distance_0', 'bids_distance_1', 'bids_distance_2', 'bids_distance_3', 'bids_distance_4', 'bids_distance_5'] ...


## 2. Feature Engineering

We construct **26 microstructure features** from the raw LOB data, organized into 5 categories:

### Original Features (11):
1. **spread_pct**: Relative spread (spread / midpoint) - liquidity cost
2. **BidDepth_k / AskDepth_k**: Total notional depth across top K levels
3. **DepthImb_k**: Order book skew (bid-ask depth imbalance)
4. **MO_bid_tau / MO_ask_tau**: Market order notional (aggressive flow)
5. **MO_imb_tau**: Market order imbalance (buying vs selling pressure)
6. **CA_bid_tau / CA_ask_tau**: Cancellation notional (strategic removals)
7. **CA_imb_tau**: Cancel imbalance (toxicity signal)
8. **rv_5s**: Realized volatility (5-second rolling std of returns)

### New Features for Accuracy Boost (15):

**Momentum Features (5):**
- **ret_5s, ret_10s, ret_30s, ret_60s**: Multi-timeframe returns (trend direction)
- **ret_accel**: Price acceleration (momentum of momentum)

**Volatility Features (3):**
- **rv_30s**: 30-second realized volatility
- **rv_ratio**: Short-term vs long-term volatility ratio (detects regime changes)
- **hl_range_10s**: High-low range as % of midpoint (price variability)

**Flow Momentum (3):**
- **MO_imb_chg**: Change in market order imbalance (flow acceleration)
- **CA_imb_chg**: Change in cancel imbalance (toxicity acceleration)
- **Depth_imb_chg**: Change in depth imbalance (book shift dynamics)

**Order Book Shape (2):**
- **depth_concentration**: Top 3 levels / Total depth (liquidity distribution)
- **total_depth**: Log of total depth (overall liquidity level)

**Temporal Features (2):**
- **hour_sin / hour_cos**: Cyclical hour encoding (intraday patterns)

### Rationale:
- **Momentum**: Captures trend and mean-reversion signals
- **Volatility**: Different regimes have different volatility profiles
- **Flow changes**: Acceleration matters more than absolute levels
- **Shape**: Concentrated vs dispersed liquidity affects predictability
- **Temporal**: Crypto markets have hour-of-day effects (Asian/European/US trading hours)

In [ ]:
def level_cols(prefix, k=TOP_K, in_df=None):
    cols = [f"{prefix}_{i}" for i in range(k)]
    if in_df is not None:
        cols = [c for c in cols if c in in_df.columns]
    return cols

def agg_levels(df, prefix, k=TOP_K):
    cols = level_cols(prefix, k, in_df=df)
    if not cols:
        return pd.Series(0.0, index=df.index)
    return df[cols].sum(axis=1)

# ============================================================================
# ORIGINAL FEATURES (11 features)
# ============================================================================

# Spread
df['spread_pct'] = (df[SPREAD_COL] / df[MID_COL]).replace([np.inf, -np.inf], np.nan).fillna(0)

# Depth features
df['BidDepth_k'] = agg_levels(df, "bids_limit_notional", TOP_K)
df['AskDepth_k'] = agg_levels(df, "asks_limit_notional", TOP_K)
df['DepthImb_k'] = (df['BidDepth_k'] - df['AskDepth_k']) / (df['BidDepth_k'] + df['AskDepth_k'] + 1e-9)

# Market order flows
df['MO_bid_tau'] = agg_levels(df, "bids_market_notional", TOP_K)
df['MO_ask_tau'] = agg_levels(df, "asks_market_notional", TOP_K)
df['MO_imb_tau'] = (df['MO_ask_tau'] - df['MO_bid_tau']) / (df['MO_ask_tau'] + df['MO_bid_tau'] + 1e-9)

# Cancel flows
df['CA_bid_tau'] = agg_levels(df, "bids_cancel_notional", TOP_K)
df['CA_ask_tau'] = agg_levels(df, "asks_cancel_notional", TOP_K)
df['CA_imb_tau'] = (df['CA_ask_tau'] - df['CA_bid_tau']) / (df['CA_ask_tau'] + df['CA_bid_tau'] + 1e-9)

# Realized volatility
ret = df[MID_COL].pct_change()
df['rv_5s'] = ret.rolling(5, min_periods=1).std().fillna(0)

# ============================================================================
# NEW FEATURES FOR ACCURACY BOOST (15 additional features)
# ============================================================================

print("Engineering additional features for improved accuracy...")

# --- 1. MOMENTUM FEATURES (5 features) ---
# Multi-timeframe returns capture trend direction
df['ret_5s'] = df[MID_COL].pct_change(5).fillna(0)
df['ret_10s'] = df[MID_COL].pct_change(10).fillna(0)
df['ret_30s'] = df[MID_COL].pct_change(30).fillna(0)
df['ret_60s'] = df[MID_COL].pct_change(60).fillna(0)

# Price acceleration (second derivative of price)
df['ret_accel'] = df['ret_5s'] - df['ret_5s'].shift(5).fillna(0)

# --- 2. VOLATILITY FEATURES (3 features) ---
# Volatility ratios capture regime shifts
df['rv_30s'] = ret.rolling(30, min_periods=1).std().fillna(0)
df['rv_ratio'] = (df['rv_5s'] / (df['rv_30s'] + 1e-9)).replace([np.inf, -np.inf], 0).fillna(0)

# High-low range (price variability)
df['hl_range_10s'] = df[MID_COL].rolling(10).max() - df[MID_COL].rolling(10).min()
df['hl_range_10s'] = (df['hl_range_10s'] / df[MID_COL]).fillna(0)

# --- 3. FLOW MOMENTUM (3 features) ---
# Changes in imbalances indicate flow acceleration
df['MO_imb_chg'] = df['MO_imb_tau'] - df['MO_imb_tau'].shift(5).fillna(0)
df['CA_imb_chg'] = df['CA_imb_tau'] - df['CA_imb_tau'].shift(5).fillna(0)
df['Depth_imb_chg'] = df['DepthImb_k'] - df['DepthImb_k'].shift(5).fillna(0)

# --- 4. ORDER BOOK SHAPE (2 features) ---
# Depth concentration: how much liquidity is in top levels vs deeper levels
top3_bid = agg_levels(df, "bids_limit_notional", 3)
top3_ask = agg_levels(df, "asks_limit_notional", 3)
df['depth_concentration'] = ((top3_bid + top3_ask) / (df['BidDepth_k'] + df['AskDepth_k'] + 1e-9)).fillna(0)

# Total depth (proxy for liquidity)
df['total_depth'] = np.log1p(df['BidDepth_k'] + df['AskDepth_k'])

# --- 5. TEMPORAL FEATURES (2 features) ---
# Crypto markets have intraday patterns
df['hour'] = pd.to_datetime(df[TIME_COL]).dt.hour
df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)  # Cyclical encoding
df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

# Drop temporary column
df = df.drop('hour', axis=1)

# ============================================================================
# FEATURE LIST (26 total features: 11 original + 15 new)
# ============================================================================

feature_cols = [
    # Original features (11)
    'spread_pct', 'DepthImb_k', 'MO_imb_tau', 'CA_imb_tau', 'rv_5s',
    'BidDepth_k', 'AskDepth_k', 'MO_bid_tau', 'MO_ask_tau', 'CA_bid_tau', 'CA_ask_tau',
    
    # Momentum features (5)
    'ret_5s', 'ret_10s', 'ret_30s', 'ret_60s', 'ret_accel',
    
    # Volatility features (3)
    'rv_30s', 'rv_ratio', 'hl_range_10s',
    
    # Flow momentum (3)
    'MO_imb_chg', 'CA_imb_chg', 'Depth_imb_chg',
    
    # Order book shape (2)
    'depth_concentration', 'total_depth',
    
    # Temporal features (2)
    'hour_sin', 'hour_cos'
]

print(f"\nTotal features: {len(feature_cols)}")
print("\nFeature existence check:")
print(pd.DataFrame({'exists': [c in df.columns for c in feature_cols]}, index=feature_cols))

## 3. Label Creation (Binary Classification: Up vs Down)

We create **binary classification labels** based on the **forward 30-second mid-price return**.

### Binary Classification:
- **y = +1** (UP): if (mid_t+30s - mid_t) / mid_t > THRESH_BPS
- **y = -1** (DOWN): if (mid_t+30s - mid_t) / mid_t < -THRESH_BPS
- **Samples with |return| < THRESH_BPS are FILTERED OUT** (too small to predict reliably)

### Why Binary (not Ternary)?
- **Simpler problem:** Up vs Down is easier than Up/Down/Flat
- **Higher accuracy expected:** 50% random baseline → easier to beat than 33.3%
- **More practical:** Trading strategies care about direction, not "flat"
- **Cleaner signals:** Removes noisy near-zero returns

### Threshold: THRESH_BPS = {THRESH_BPS}bp
- Filters out microstructure noise
- Only keeps economically meaningful moves
- Higher threshold = fewer but cleaner labels

### Label Distribution:
We expect roughly balanced up/down labels (close to 50/50).

In [ ]:
df = df.sort_values(TIME_COL).reset_index(drop=True)
approx_dt = (df[TIME_COL].iloc[1] - df[TIME_COL].iloc[0]).total_seconds()
shift_n = max(1, int(round(HORIZON_SEC / max(1, approx_dt))))
df['mid_fwd'] = df[MID_COL].shift(-shift_n)

thresh = THRESH_BPS * 1e-4
r = (df['mid_fwd'] - df[MID_COL]) / df[MID_COL]

# Binary classification: +1 for up, -1 for down, NaN for small moves
y = np.where(r > thresh, 1, np.where(r < -thresh, -1, np.nan))
df['y'] = y

# Filter out small moves (keep only clear up/down)
df_lbl = df.dropna(subset=['mid_fwd', 'y']).copy()

# Check label distribution
label_counts = df_lbl['y'].value_counts(normalize=True).to_dict()
print(f"Label distribution (binary): {label_counts}")
print(f"Up samples:   {label_counts.get(1.0, 0):.1%}")
print(f"Down samples: {label_counts.get(-1.0, 0):.1%}")
print(f"Total samples after filtering: {len(df_lbl):,} (removed {len(df) - len(df_lbl):,} small moves)")

## 4. Feature Scaling & PCA

### Standardization
All features are z-scored (mean=0, std=1) using `StandardScaler`. This is critical because:
- Features have different scales (e.g., spread_pct ~ 0.001, BidDepth_k ~ 100,000)
- ML algorithms (especially Logistic Regression and HMM) are sensitive to scale

**Important**: We fit the scaler on training data only, then transform both train and test. This prevents data leakage.

### PCA (Principal Component Analysis)
We apply PCA to reduce dimensionality from **26 features** to ~**12-15 components** while retaining 95% of variance. Benefits:
- **Reduces noise**: Minor components often contain noise rather than signal
- **Improves generalization**: Fewer parameters reduce overfitting risk
- **Faster training**: Especially important for HMM (O(n² components))
- **Removes multicollinearity**: Our 26 features are correlated; PCA creates independent components

**Note**: PCA is fit on training data only to prevent leakage.

In [6]:
from sklearn.preprocessing import StandardScaler

def scale_features(df_train, df_test, cols=None, use_pca=USE_PCA, pca_var=PCA_VARIANCE):
    if cols is None:
        cols = feature_cols
    scaler = StandardScaler()
    Xtr = scaler.fit_transform(df_train[cols].fillna(0.0).values)
    Xte = scaler.transform(df_test[cols].fillna(0.0).values)
    
    pca = None
    if use_pca:
        pca = PCA(n_components=pca_var, random_state=RANDOM_STATE)
        Xtr = pca.fit_transform(Xtr)
        Xte = pca.transform(Xte)
    
    return Xtr, Xte, pca

## 5. Walk-Forward Time-Series Cross-Validation

### Why Walk-Forward?
Standard k-fold CV shuffles data randomly, which **leaks future information** in time-series contexts. Walk-forward validation simulates real trading:
- Train on past data only
- Test on strictly future data
- Embargo period prevents train/test overlap

### Configuration:
- **Training window**: 3 days of historical data
- **Test window**: 4 hours of future data
- **Embargo**: 60 seconds gap between train and test sets
- **Stride**: 12 hours between folds (creates 9 overlapping folds)

### Why 60s embargo?
Prevents near-term autocorrelation from contaminating test set. In high-frequency data, observations 1 second apart are highly correlated.

### Fold Structure:
```
Fold 0: Train [Day 0-3] → Test [Day 3 + 60s to Day 3 + 4hr]
Fold 1: Train [Day 0.5-3.5] → Test [Day 3.5 + 60s to Day 3.5 + 4hr]
...
```

In [7]:

def walk_forward_splits_hours_v2(
    timestamps, train_days=3, test_hours=4, embargo_sec=60, stride_hours=None
):
    ts = pd.to_datetime(timestamps, utc=True).sort_values()
    tmin = ts.min()
    tmax = ts.max()
    stride = pd.Timedelta(hours=(stride_hours or test_hours))

    cur = tmin
    while True:
        tr_s = cur
        tr_e = tr_s + pd.Timedelta(days=train_days)
        te_s = tr_e + pd.Timedelta(seconds=embargo_sec)
        te_e = te_s + pd.Timedelta(hours=test_hours)
        if te_e > tmax:
            break
        yield (tr_s, tr_e, te_s, te_e)
        cur = cur + stride


## 6. Regime Discovery via Hidden Markov Model (HMM)

### Unsupervised Learning: Identifying Market Regimes

We use a **2-state Gaussian HMM** to discover latent market regimes from the PCA-transformed features. The HMM models:
- **Hidden states**: Latent regime variable (balanced vs. toxic)
- **Emissions**: Observed features (PCA components)
- **Transitions**: Regime persistence and switching dynamics

### Why HMM instead of K-Means?
- **Temporal dynamics**: HMM captures regime persistence (toxic periods cluster in time)
- **Probabilistic**: Soft assignments rather than hard clusters
- **Markov property**: Current state depends on previous state (realistic for markets)

### Identifying the "Toxic" State
After fitting the HMM, we label states by computing a **toxicity score** for each state:
```
toxicity_score = |MO_imb_tau| + |CA_imb_tau| + |DepthImb_k|
```

The state with **higher average toxicity score** is labeled as the "toxic" regime.

### Hyperparameters:
- **n_components=2**: Binary regime (balanced vs. toxic)
- **covariance_type='diag'**: Assumes feature independence (faster, works well in practice)
- **n_iter=150**: EM algorithm iterations for convergence

In [ ]:
def fit_hmm(X_train, df_train, n_states=2, random_state=42):
    """
    Fit a Gaussian HMM to discover market regimes and identify the toxic state.
    
    Parameters:
    -----------
    X_train : array-like, shape (n_samples, n_features)
        Scaled and PCA-transformed features
    df_train : DataFrame
        Original training data (needed for toxicity score calculation)
    n_states : int
        Number of hidden states (default=2 for balanced/toxic)
    random_state : int
        Random seed for reproducibility
        
    Returns:
    --------
    hmm : GaussianHMM
        Fitted HMM model
    tox_state : int
        Index of the state identified as "toxic" (0 or 1)
    """
    # Compute toxicity score for each sample
    tox_score_cols = ['MO_imb_tau', 'CA_imb_tau', 'DepthImb_k']
    train_score = df_train[tox_score_cols].fillna(0).copy()
    train_score['tox_score'] = (
        train_score['MO_imb_tau'].abs() +
        train_score['CA_imb_tau'].abs() +
        train_score['DepthImb_k'].abs()
    )
    
    # Fit HMM on PCA-transformed features
    hmm = GaussianHMM(
        n_components=n_states,
        covariance_type=HMM_COVARIANCE,
        n_iter=HMM_ITERATIONS,
        random_state=random_state
    )
    
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", message="Model is not converging")
        hmm.fit(X_train)
    
    # Predict states and compute average toxicity score per state
    states = hmm.predict(X_train)
    state_toxicity = pd.DataFrame({
        'state': states,
        'tox': train_score['tox_score'].values
    }).groupby('state')['tox'].mean()
    
    # Identify toxic state as the one with higher average toxicity
    tox_state = int(state_toxicity.idxmax())
    
    return hmm, tox_state

## 7. Supervised Learning: Baseline vs. Regime-Aware Models

We train **four models** per fold to evaluate the regime-awareness hypothesis:

### Baseline Models (no regime information):
1. **Logistic Regression**: Linear model, fast, interpretable
2. **XGBoost**: Gradient-boosted trees, captures non-linearities

### Regime-Aware Models (with HMM regime as feature):
3. **Logistic Regression + Regime**: LR with regime binary feature added
4. **XGBoost + Regime**: XGBoost with regime binary feature added

### Model Configuration:
- **Class balancing**: `class_weight='balanced'` and sample weights to handle class imbalance
- **Early stopping**: XGBoost stops if validation score plateaus for 30 rounds
- **Probabilistic outputs**: All models produce P(up), P(down), P(flat)

### Hypothesis Testing:
If regime-awareness helps, we expect:
- **Regime-aware XGBoost** > **Baseline XGBoost** (overall accuracy)
- **Hit Rate in toxic regime** > **Hit Rate in balanced regime** (regime-conditioned performance)

In [ ]:
# Binary classification: -1 (down) and +1 (up)
CLASSES = [-1, 1]
ENC_MAP = {-1: 0, 1: 1}  # Encode to 0/1 for sklearn
DEC_MAP = {v: k for k, v in ENC_MAP.items()}

def metrics_from_proba_encoded(proba, y_true_enc):
    """Compute accuracy and F1 for binary classification."""
    yhat_enc = proba.argmax(axis=1)
    yhat = np.array([DEC_MAP[i] for i in yhat_enc])
    ytrue = np.array([DEC_MAP[i] for i in y_true_enc])
    acc = accuracy_score(ytrue, yhat)
    f1 = f1_score(ytrue, yhat, average='binary', pos_label=1)
    return acc, f1, yhat, ytrue

def regime_hits(yhat, ytrue, regime_flag):
    """Compute hit rates conditioned on regime."""
    bal = (regime_flag == 0)
    tox = (regime_flag == 1)
    hr_bal = float((yhat[bal] == ytrue[bal]).mean()) if bal.any() else np.nan
    hr_tox = float((yhat[tox] == ytrue[tox]).mean()) if tox.any() else np.nan
    return hr_bal, hr_tox

df_sup = df_lbl.dropna(subset=['y']).copy().sort_values(TIME_COL)
folds = list(walk_forward_splits_hours_v2(df_sup[TIME_COL], TRAIN_DAYS, TEST_HOURS, EMBARGO_SEC, STRIDE_HRS))
print("Planned folds:", len(folds))
print(f"Binary classification: {len(CLASSES)} classes (Up vs Down)")
print(f"Random baseline: 50.0%\n")
for i, (a, b, c, d) in enumerate(folds[:5]):
    print(f"Fold {i}: TRAIN {a}→{b}  TEST {c}→{d}")

## 8. Walk-Forward Evaluation Loop

This section executes the complete pipeline for each fold:
1. Split data into train/test based on time windows
2. Scale features and apply PCA (fit on train, transform both)
3. Train baseline models (LR and XGBoost without regime)
4. Fit HMM to discover regimes
5. Augment features with regime indicator
6. Train regime-aware models (LR and XGBoost with regime)
7. Compute metrics: accuracy, F1, regime-conditioned hit rates

### Key Metrics:
- **Accuracy**: Overall correctness (% correct predictions)
- **F1 Score**: Harmonic mean of precision and recall (macro-averaged across 3 classes)
- **Hit Rate (Balanced)**: Accuracy when HMM predicts balanced regime
- **Hit Rate (Toxic)**: Accuracy when HMM predicts toxic regime
- **Lift**: Hit Rate(toxic) - Hit Rate(balanced) — **THIS IS THE KEY METRIC**

### Expected Outcome:
**Lift > 0** validates our hypothesis that toxic regimes are more predictable.

In [ ]:
import time

results = []
fold = 0
MIN_TRAIN = 300
MIN_TEST  = 100

for (tr_s, tr_e, te_s, te_e) in folds:
    fold_start = time.time()
    train = df_sup[(df_sup[TIME_COL] >= tr_s) & (df_sup[TIME_COL] < tr_e)].copy()
    test  = df_sup[(df_sup[TIME_COL] >= te_s) & (df_sup[TIME_COL] < te_e)].copy()
    if len(train) < MIN_TRAIN or len(test) < MIN_TEST:
        continue

    # Subsample training data if too large (computational efficiency)
    if len(train) > MAX_TRAIN_SAMPLES:
        train = train.sample(n=MAX_TRAIN_SAMPLES, random_state=RANDOM_STATE)
        print(f"\n[Fold {fold+1}/{len(folds)}] Train: {len(train):,} samples (subsampled), Test: {len(test):,} samples")
    else:
        print(f"\n[Fold {fold+1}/{len(folds)}] Train: {len(train):,} samples, Test: {len(test):,} samples")
    
    # Step 1: Scale features and apply PCA
    t0 = time.time()
    Xtr_base, Xte_base, pca_model = scale_features(train, test)
    if USE_PCA and pca_model is not None and fold == 0:
        n_components = pca_model.n_components_
        print(f"PCA: {len(feature_cols)} features → {n_components} components (retained {PCA_VARIANCE*100}% variance)")
    print(f"  Scaling+PCA: {time.time()-t0:.1f}s")
    
    # Encode labels for sklearn (binary: 0 and 1)
    ytr_enc = np.array([ENC_MAP[v] for v in train['y'].values])
    yte_enc = np.array([ENC_MAP[v] for v in test['y'].values])

    # Step 2: Train baseline Logistic Regression (no regime info)
    t0 = time.time()
    lr_base = LogisticRegression(max_iter=1000, C=0.5, class_weight='balanced', solver='lbfgs')
    lr_base.fit(Xtr_base, ytr_enc)
    acc_lr_base, f1_lr_base, _, _ = metrics_from_proba_encoded(lr_base.predict_proba(Xte_base), yte_enc)
    print(f"  LR baseline: {time.time()-t0:.1f}s")

    # Step 3: Train baseline XGBoost (no regime info) - BINARY
    t0 = time.time()
    w_base = compute_sample_weight(class_weight='balanced', y=ytr_enc)
    xgb_base = XGBClassifier(
        n_estimators=XGB_N_ESTIMATORS,
        max_depth=XGB_MAX_DEPTH,
        learning_rate=XGB_LEARNING_RATE,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='binary:logistic',  # Changed from multi:softprob
        tree_method='hist',
        reg_lambda=1.0,
        random_state=RANDOM_STATE,
        early_stopping_rounds=XGB_EARLY_STOPPING
    )
    # Use 80/20 split for early stopping validation
    split_idx = int(0.8 * len(Xtr_base))
    xgb_base.fit(
        Xtr_base[:split_idx], ytr_enc[:split_idx],
        sample_weight=w_base[:split_idx],
        eval_set=[(Xtr_base[split_idx:], ytr_enc[split_idx:])],
        verbose=False
    )
    acc_xgb_base, f1_xgb_base, _, _ = metrics_from_proba_encoded(xgb_base.predict_proba(Xte_base), yte_enc)
    print(f"  XGB baseline: {time.time()-t0:.1f}s")

    # Step 4: Fit HMM to discover regimes
    t0 = time.time()
    hmm, tox_state = fit_hmm(Xtr_base, train, n_states=2)
    print(f"  HMM: {time.time()-t0:.1f}s")
    
    # Step 5: Predict regimes for train and test
    s_tr = hmm.predict(Xtr_base)
    s_te = hmm.predict(Xte_base)

    # Binary regime indicator: 1 = toxic, 0 = balanced
    train['regime_hmm'] = (s_tr == tox_state).astype(int)
    test['regime_hmm']  = (s_te == tox_state).astype(int)

    # Step 6: Augment features with regime indicator
    Xtr = np.c_[Xtr_base, train['regime_hmm'].values]
    Xte = np.c_[Xte_base, test['regime_hmm'].values]

    # Step 7: Train regime-aware Logistic Regression
    t0 = time.time()
    lr = LogisticRegression(max_iter=1000, C=0.5, class_weight='balanced', solver='lbfgs')
    lr.fit(Xtr, ytr_enc)
    acc_lr, f1_lr, yhat_lr, ytrue_lr = metrics_from_proba_encoded(lr.predict_proba(Xte), yte_enc)
    print(f"  LR regime-aware: {time.time()-t0:.1f}s")

    # Step 8: Train regime-aware XGBoost - BINARY
    t0 = time.time()
    w = compute_sample_weight(class_weight='balanced', y=ytr_enc)
    xgb = XGBClassifier(
        n_estimators=XGB_N_ESTIMATORS,
        max_depth=XGB_MAX_DEPTH,
        learning_rate=XGB_LEARNING_RATE,
        subsample=0.8,
        colsample_bytree=0.8,
        objective='binary:logistic',  # Changed from multi:softprob
        tree_method='hist',
        reg_lambda=1.0,
        random_state=RANDOM_STATE,
        early_stopping_rounds=XGB_EARLY_STOPPING
    )
    split_idx = int(0.8 * len(Xtr))
    xgb.fit(
        Xtr[:split_idx], ytr_enc[:split_idx],
        sample_weight=w[:split_idx],
        eval_set=[(Xtr[split_idx:], ytr_enc[split_idx:])],
        verbose=False
    )
    acc_xgb, f1_xgb, yhat_main, ytrue_main = metrics_from_proba_encoded(xgb.predict_proba(Xte), yte_enc)
    print(f"  XGB regime-aware: {time.time()-t0:.1f}s")

    # Step 9: Compute regime-conditioned hit rates
    hr_bal, hr_tox = regime_hits(yhat_main, ytrue_main, test['regime_hmm'].values)

    # Step 10: Store results for this fold
    results.append({
        'fold': fold,
        'test_start': te_s, 
        'test_end': te_e,
        'acc_lr_base': acc_lr_base,   
        'f1_lr_base': f1_lr_base,
        'acc_xgb_base': acc_xgb_base, 
        'f1_xgb_base': f1_xgb_base,
        'acc_lr': acc_lr,   
        'f1_lr': f1_lr,
        'acc_xgb': acc_xgb, 
        'f1_xgb': f1_xgb,
        'hr_bal': hr_bal, 
        'hr_tox': hr_tox,
        'lift': hr_tox - hr_bal,
        'pct_toxic_test': float((test['regime_hmm'] == 1).mean())
    })
    
    print(f"  FOLD TOTAL: {time.time()-fold_start:.1f}s")
    fold += 1

# Create results dataframe
df_res = pd.DataFrame(results)
print("\n" + "="*60)
print(f"EVALUATION COMPLETE: {len(df_res)} folds")
print("="*60)

if len(df_res) == 0:
    raise RuntimeError("No folds evaluated. Check TRAIN_DAYS/TEST_HOURS/STRIDE_HRS parameters.")

# Display per-fold results
cols = ['fold', 'test_start', 'test_end', 'acc_lr_base', 'acc_lr', 'acc_xgb_base', 'acc_xgb', 'hr_bal', 'hr_tox', 'lift', 'pct_toxic_test']
print("\nPer-Fold Results:")
display(df_res[cols].round(3))

# Display average metrics across folds
print("\nAverage Metrics Across Folds:")
avg_metrics = df_res[['acc_lr_base', 'acc_lr', 'acc_xgb_base', 'acc_xgb', 'hr_bal', 'hr_tox', 'lift']].mean()
display(avg_metrics.round(3))

## 9. Results Visualization

This plot shows the **regime-conditioned hit rates** for each fold, which is the central result validating our hypothesis.

### Interpretation:
- **Blue bars (Balanced regime)**: Accuracy when HMM identifies balanced market conditions
- **Orange bars (Toxic regime)**: Accuracy when HMM identifies toxic market conditions
- **Lift (key metric)**: Difference between toxic and balanced hit rates

### Expected Pattern:
If our hypothesis is correct, orange bars should be consistently higher than blue bars across most folds, indicating that **toxic regimes are more predictable** than balanced regimes.

In [ ]:
# Create figure with adequate size
fig, ax = plt.subplots(figsize=(10, 5))

# Bar positions
x = np.arange(len(df_res))
width = 0.35

# Plot bars
bars1 = ax.bar(x - width/2, df_res['hr_bal'], width, label='Balanced Regime', alpha=0.8, color='#3498db')
bars2 = ax.bar(x + width/2, df_res['hr_tox'], width, label='Toxic Regime', alpha=0.8, color='#e74c3c')

# Formatting
labels = pd.to_datetime(df_res['test_start']).dt.strftime('%m-%d %H:%M')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=45, ha='right')
ax.set_xlabel("Test Window Start Time", fontsize=11)
ax.set_ylabel("Hit Rate (Accuracy)", fontsize=11)

# Calculate mean lift
mean_lift = df_res['lift'].mean()
title_color = 'green' if mean_lift > 0 else 'red'
ax.set_title(f"Regime-Conditioned Hit Rate Across Folds\nMean Lift = {mean_lift:.3f} ({mean_lift*100:.1f}%)", 
             fontsize=12, fontweight='bold')

# Add horizontal line at 50% (random guess baseline for BINARY classification)
ax.axhline(y=0.50, color='gray', linestyle='--', linewidth=1, alpha=0.5, label='Random Baseline (50%)')

# Legend
ax.legend(loc='upper left', fontsize=10)

# Grid for readability
ax.grid(axis='y', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

# Print summary statistics
print("\nSummary Statistics:")
print(f"Mean Balanced Hit Rate: {df_res['hr_bal'].mean():.3f} ({df_res['hr_bal'].mean()*100:.1f}%)")
print(f"Mean Toxic Hit Rate:    {df_res['hr_tox'].mean():.3f} ({df_res['hr_tox'].mean()*100:.1f}%)")
print(f"Mean Lift:              {mean_lift:.3f} ({mean_lift*100:.1f}%)")
print(f"Random Baseline:        0.500 (50.0%)")
print(f"\nHypothesis Validation: {'✓ SUPPORTED' if mean_lift > 0 else '✗ NOT SUPPORTED'}")
if mean_lift > 0:
    print(f"  → Toxic regimes are {mean_lift*100:.1f}% more predictable than balanced regimes")
else:
    print(f"  → No evidence that toxic regimes are more predictable")

## 10. Baseline vs Regime-Aware Performance Comparison

This visualization compares model performance **with and without** regime information to quantify the value-add of the HMM regime discovery step.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# --- Left Plot: Logistic Regression ---
ax1 = axes[0]
x = np.arange(len(df_res))
width = 0.35

bars1 = ax1.bar(x - width/2, df_res['acc_lr_base'], width, label='LR Baseline', alpha=0.8, color='#95a5a6')
bars2 = ax1.bar(x + width/2, df_res['acc_lr'], width, label='LR + Regime', alpha=0.8, color='#3498db')

ax1.set_xlabel('Fold', fontsize=11)
ax1.set_ylabel('Accuracy', fontsize=11)
ax1.set_title(f'Logistic Regression\nMean Improvement: {(df_res["acc_lr"].mean() - df_res["acc_lr_base"].mean())*100:.2f}%', 
              fontsize=12, fontweight='bold')
ax1.set_xticks(x)
ax1.set_xticklabels([f'F{i}' for i in df_res['fold']])
ax1.axhline(y=0.50, color='red', linestyle='--', linewidth=1, alpha=0.5, label='Random (50%)')
ax1.legend(fontsize=9)
ax1.grid(axis='y', alpha=0.3, linestyle='--')

# --- Right Plot: XGBoost ---
ax2 = axes[1]
bars3 = ax2.bar(x - width/2, df_res['acc_xgb_base'], width, label='XGB Baseline', alpha=0.8, color='#95a5a6')
bars4 = ax2.bar(x + width/2, df_res['acc_xgb'], width, label='XGB + Regime', alpha=0.8, color='#e74c3c')

ax2.set_xlabel('Fold', fontsize=11)
ax2.set_ylabel('Accuracy', fontsize=11)
ax2.set_title(f'XGBoost\nMean Improvement: {(df_res["acc_xgb"].mean() - df_res["acc_xgb_base"].mean())*100:.2f}%', 
              fontsize=12, fontweight='bold')
ax2.set_xticks(x)
ax2.set_xticklabels([f'F{i}' for i in df_res['fold']])
ax2.axhline(y=0.50, color='red', linestyle='--', linewidth=1, alpha=0.5, label='Random (50%)')
ax2.legend(fontsize=9)
ax2.grid(axis='y', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()

# Print improvement statistics
print("\nRegime-Awareness Performance Gains:")
print("="*50)
lr_improvement = (df_res['acc_lr'].mean() - df_res['acc_lr_base'].mean()) * 100
xgb_improvement = (df_res['acc_xgb'].mean() - df_res['acc_xgb_base'].mean()) * 100

print(f"Logistic Regression:")
print(f"  Baseline:      {df_res['acc_lr_base'].mean():.4f} ({df_res['acc_lr_base'].mean()*100:.2f}%)")
print(f"  + Regime:      {df_res['acc_lr'].mean():.4f} ({df_res['acc_lr'].mean()*100:.2f}%)")
print(f"  Improvement:   {lr_improvement:+.2f}%")
print(f"\nXGBoost:")
print(f"  Baseline:      {df_res['acc_xgb_base'].mean():.4f} ({df_res['acc_xgb_base'].mean()*100:.2f}%)")
print(f"  + Regime:      {df_res['acc_xgb'].mean():.4f} ({df_res['acc_xgb'].mean()*100:.2f}%)")
print(f"  Improvement:   {xgb_improvement:+.2f}%")
print(f"\nRandom Baseline: 50.00%")

## 11. Feature Importance Analysis

XGBoost tracks which features contribute most to prediction accuracy. This analysis reveals:
1. Which PCA components are most predictive
2. Whether the HMM regime feature is actually being used by the model
3. Feature importance validation (confirms our feature engineering is meaningful)

In [ ]:
# Train a final model for feature importance analysis
print("Training final XGBoost model on all data for feature importance analysis...")

# Use all labeled data
df_all = df_lbl.dropna(subset=['y']).copy()

# Take a sample to keep training time reasonable
sample_size = min(200000, len(df_all))
df_sample = df_all.sample(n=sample_size, random_state=RANDOM_STATE)

# Split 80/20 for train/val
split_idx = int(0.8 * len(df_sample))
df_train_full = df_sample.iloc[:split_idx]
df_val_full = df_sample.iloc[split_idx:]

# Scale and PCA
Xtr, Xval, pca = scale_features(df_train_full, df_val_full)

# Fit HMM
hmm_full, tox_state_full = fit_hmm(Xtr, df_train_full, n_states=2)

# Predict regimes
s_tr = hmm_full.predict(Xtr)
s_val = hmm_full.predict(Xval)

df_train_full['regime_hmm'] = (s_tr == tox_state_full).astype(int)
df_val_full['regime_hmm'] = (s_val == tox_state_full).astype(int)

# Augment with regime
Xtr_regime = np.c_[Xtr, df_train_full['regime_hmm'].values]
Xval_regime = np.c_[Xval, df_val_full['regime_hmm'].values]

# Encode labels (binary)
ytr = np.array([ENC_MAP[v] for v in df_train_full['y'].values])
yval = np.array([ENC_MAP[v] for v in df_val_full['y'].values])

# Train regime-aware XGBoost (BINARY)
w = compute_sample_weight(class_weight='balanced', y=ytr)
xgb_final = XGBClassifier(
    n_estimators=XGB_N_ESTIMATORS,
    max_depth=XGB_MAX_DEPTH,
    learning_rate=XGB_LEARNING_RATE,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',  # Binary classification
    tree_method='hist',
    reg_lambda=1.0,
    random_state=RANDOM_STATE,
    early_stopping_rounds=XGB_EARLY_STOPPING
)

xgb_final.fit(
    Xtr_regime, ytr,
    sample_weight=w,
    eval_set=[(Xval_regime, yval)],
    verbose=False
)

# Extract feature importance
feature_importance = xgb_final.feature_importances_

# Create feature names (PCA components + regime)
n_pca_components = Xtr.shape[1]
feature_names = [f'PC{i+1}' for i in range(n_pca_components)] + ['Regime']

# Sort by importance
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)

# Plot
fig, ax = plt.subplots(figsize=(10, 6))
colors = ['#e74c3c' if f == 'Regime' else '#3498db' for f in importance_df['Feature']]
bars = ax.barh(importance_df['Feature'], importance_df['Importance'], color=colors, alpha=0.8)

ax.set_xlabel('Importance (Gain)', fontsize=11)
ax.set_ylabel('Feature', fontsize=11)
ax.set_title('XGBoost Feature Importance (Regime-Aware Model, Binary Classification)', fontsize=12, fontweight='bold')
ax.grid(axis='x', alpha=0.3, linestyle='--')
ax.invert_yaxis()

plt.tight_layout()
plt.show()

# Print importance values
print("\nFeature Importance Rankings:")
print("="*50)
for idx, row in importance_df.iterrows():
    pct = row['Importance'] / feature_importance.sum() * 100
    marker = " ⭐" if row['Feature'] == 'Regime' else ""
    print(f"{row['Feature']:10s}: {row['Importance']:.4f} ({pct:5.2f}%){marker}")

regime_importance = importance_df[importance_df['Feature'] == 'Regime']['Importance'].values[0]
regime_pct = regime_importance / feature_importance.sum() * 100

print(f"\n{'='*50}")
print(f"Regime Feature Contribution: {regime_pct:.2f}%")
if regime_pct > 10:
    print("✓ Regime is a STRONG predictor (>10% importance)")
elif regime_pct > 5:
    print("✓ Regime is a MODERATE predictor (5-10% importance)")
else:
    print("⚠ Regime has LOW importance (<5%) - may not be adding much value")
    
print(f"\nNumber of PCA components: {n_pca_components}")
print(f"Variance explained: {PCA_VARIANCE*100}%")